In [2]:
import pandas as pd

## Get Col Types

In [3]:
datadict = pd.read_csv('../data/raw/WiDS Datathon 2020 Dictionary.csv')

In [4]:
# get cols

In [5]:
datadict = pd.read_csv('../data/raw/WiDS Datathon 2020 Dictionary.csv')
continuous_cols = list(
    list(datadict[datadict['Data Type'] == 'integer']['Variable Name'].unique())
    + list(datadict[datadict['Data Type'] == 'numeric']['Variable Name'].unique())
)
categorical_cols = list(datadict[datadict['Data Type'] == 'string']['Variable Name'].unique())
binary_cols = list(datadict[datadict['Data Type'] == 'binary']['Variable Name'].unique())

target_col = 'hospital_death'
binary_cols.remove(target_col)


# fix datadict variable names
categorical_cols.remove('icu_admit_type')
continuous_cols.remove('pred')

# fix missmapped continuous cols
continuous_cols.extend(['bmi', 'apache_2_diagnosis', 'apache_3j_diagnosis'])
categorical_cols = [x for x in categorical_cols if x not in ['bmi', 'apache_2_diagnosis', 'apache_3j_diagnosis']]

## Get Data

In [6]:
df = pd.read_csv('../data/raw/training_v2.csv')

In [21]:
df.shape

(91713, 186)

In [7]:
# constrain features to typings
df = df[[target_col] + continuous_cols + categorical_cols + binary_cols]

## Typify

In [8]:
# type cast

In [9]:
df[continuous_cols] = df[continuous_cols].astype('float32')

In [10]:
df[categorical_cols] = df[categorical_cols].astype('str').astype('category')

In [11]:
df[binary_cols] = df[binary_cols].astype('str').astype('category')

In [12]:
df[target_col] = df[target_col].astype('str').astype('category')

## Splitting

In [17]:
from sklearn.model_selection import train_test_split
import numpy as np

In [19]:
strat_cols = [target_col]

train = pd.DataFrame()
val = pd.DataFrame()
test = pd.DataFrame()

for hospital_id in df['hospital_id'].unique():
    subset = df[df['hospital_id'] == hospital_id]
    
    if 10 < len(subset):
        classes, y_indices = np.unique(subset[strat_cols], return_inverse=True)
        class_counts = np.bincount(y_indices)

        if 2 > np.min(class_counts):
            tmp_train, tmp_test = train_test_split(subset, test_size=0.1, random_state=0)
            tmp_train, tmp_val = train_test_split(tmp_train, test_size=0.1, random_state=0)
        else:
            tmp_train, tmp_test = train_test_split(subset, test_size=0.1, random_state=0, stratify=subset[strat_cols])
            tmp_train, tmp_val = train_test_split(tmp_train, test_size=0.1, random_state=0, stratify=tmp_train[strat_cols])

        train = train.append(tmp_train)
        val = val.append(tmp_val)
        test = test.append(tmp_test)

In [20]:
train.shape, val.shape, test.shape

((74123, 186), (8303, 186), (9230, 186))